In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-car-dataset/sample_submission.csv
/kaggle/input/used-car-dataset/train.csv
/kaggle/input/used-car-dataset/test.csv


In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv('/kaggle/input/used-car-dataset/train.csv')
df_test = pd.read_csv('/kaggle/input/used-car-dataset/test.csv')

In [3]:
# EDA of the variable: BRAND.
# EDA of the variable: Model.
# EDA of the variable: Engine.
# EDA of the variable: Transmission.
# EDA of the variable: ext_col.
# EDA of the variable: int_col
# make variables: brand_avg, model_avg, engine_avg, transmission_avg. Do not include collors for now
df_train['brand_avg'] = df_train.groupby('brand')['price'].transform('mean')
df_train['model_avg'] = df_train.groupby('model')['price'].transform('mean')
df_train['engine_avg'] = df_train.groupby('engine')['price'].transform('mean')
df_train['transmission_avg'] = df_train.groupby('transmission')['price'].transform('mean')

df_test['brand_avg'] = df_train.groupby('brand')['price'].transform('mean')
df_test['model_avg'] = df_train.groupby('model')['price'].transform('mean')
df_test['engine_avg'] = df_train.groupby('engine')['price'].transform('mean')
df_test['transmission_avg'] = df_train.groupby('transmission')['price'].transform('mean')


In [4]:
### Drop ID column
df_train.drop(columns='id', inplace=True)
df_test.drop(columns='id', inplace=True)

### Clean the 'Accident' column
df_train['accident'].unique()
df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
df_train.loc[df_train['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_train.drop(columns='accident', inplace = True)

df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False
df_test.loc[df_test['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_test.drop(columns='accident', inplace = True)

## Clean the 'clean_title' column
df_train['clean_title'].unique()
df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
df_train.drop(columns='clean_title', inplace = True)

df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True
df_test.drop(columns='clean_title', inplace = True)


In [5]:
# Define the Tables
X = df_train
X = X.drop(columns='price')
y = df_train.price

# Define the columns that are going to be used
numerical_cols = ['model_year', 'milage', 'brand_avg', 'model_avg', 'engine_avg', 'transmission_avg']
categorical_cols = ['fuel_type', 'cleanbool', 'accidentbool']

# Separate test and validation of the df_train dataframe
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Defining strategies for numerical and categorical columns
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle the preprocessor

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



In [6]:
# Modeling
model = RandomForestRegressor(n_estimators = 100, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('model', model)
                 ])



In [7]:
clf.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['model_year', 'milage',
                                                   'brand_avg', 'model_avg',
                                                   'engine_avg',
                                                   'transmission_avg']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['fuel_type', 'cleanbool',
                                                   'accidentbool'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [8]:
preds = clf.predict(X_valid)
print(np.sqrt(mean_squared_error(preds, y_valid)))

73983.43090651305


In [9]:
## Run model and make it spit the submissioN!
X = df_test
prediccion = clf.predict(X)

submission = pd.DataFrame()
temp = pd.read_csv('/kaggle/input/used-car-dataset/sample_submission.csv')
submission['id'] = temp['id']
submission['price'] = prediccion
submission.to_csv('submission.csv',index=False)